In [3]:
import torch
import torch.nn as nn
from torch.nn import functional as F
import tiktoken

In [31]:
class BobNet(nn.Module):

    def __init__(self):
        super().__init__()
        self.encoding = tiktoken.get_encoding("r50k_base")
        self.emb_size = self.encoding.n_vocab
        self.emb_channels = 32

        print(self.emb_size)
        print(self.emb_channels)

        self.emb = nn.Embedding(self.emb_size, self.emb_channels)

    def forward(self, x):
        x = self.encoding.encode(x)
        x = self.emb(torch.tensor(x))
        return x
    

sup = BobNet()
sup('tiktoken is goated')

50257
32
torch.Size([6, 32])


tensor([[-0.1962, -0.0137,  0.0802,  0.1040,  2.0837, -0.7427,  0.3677,  0.1969,
         -0.2545,  0.0431,  1.0508, -0.6777,  0.3638,  0.6251,  1.1940, -0.5760,
          1.1642,  0.6589, -1.4920,  1.9627,  0.1864, -0.2340,  0.0910,  1.8666,
         -0.2770, -0.3657, -1.0958,  1.3936, -0.0818,  0.8016,  1.1561, -1.2809],
        [-1.7694,  1.0676, -0.1075,  0.2333, -0.8929, -0.4410,  0.0763, -0.0733,
          0.9464,  0.9393,  0.7507, -1.4495, -0.5691,  0.5397,  1.3439,  0.0095,
         -1.2647, -1.6720,  1.5544, -1.4235,  1.7336, -1.2097, -0.8614, -0.4005,
          0.9290, -0.3606, -0.3328, -0.8599, -0.8436,  0.4122, -1.0227,  0.1592],
        [-0.7621,  0.3943,  0.4253,  1.9987,  0.6538, -1.2100, -0.4145,  0.6854,
          0.3204,  0.0754,  1.7583,  0.1327, -2.4148, -0.5442, -1.1585, -0.9592,
         -1.5059,  0.7428,  1.6771,  0.8391, -1.0323,  0.5918, -0.7519, -0.1122,
         -2.0282,  0.4637,  0.9534, -0.5927,  0.1276,  1.4806,  0.1294, -0.1742],
        [-0.2692, -1.0386